# <center>Missão Padrão Delta</center>

## <center><u>``Resumo do código``</u></center>

>### Entra: 
> - 1_Ninjas
> - 2_Delta  
>### Sai: 
> - Long_DataFusion
> - Wide_DataFusion
> - Wide_DataFusion_2022
> - G_Classic
> - G_Classic_Reposto
> - G_Classic_SemReposto
> - G_Rotura
> - G_NumProdutos
---
O objectivo é receber dados dos Ninjas e da Delta e devolver ficheiros prontos para fazer os Dashboards. 


---


In [225]:
import pandas as pd
import numpy as np
import datetime
import json
from ipywidgets import interact

def escrever_excel(dfa, nome):
    dfa.to_excel(f'D:\\B&N Dados\\Delta\\Padrão\\Padrão_08_2023\\{nome}.xlsx', index=False)
    
def escrever_csv(dfa, nome):
    dfa.to_csv(f'D:\\B&N Dados\\Delta\\Padrão\\Padrão_08_2023\\{nome}.csv', index=False)
    
def escrever_txt(dfa, nome):
    dfa.to_csv(f'D:\\B&N Dados\\Delta\\Padrão\\Padrão_08_2023\\{nome}.txt', sep='\t', index=False, header=False)
    
def ler_json(ficheiro):
    with open(f'D:\\B&N Dados\\Delta\\{ficheiro}.json', 'r') as file:
        mapa = json.load(file)
    return mapa

<a id="section0"></a>

# Índice

1. [Data Cleaning](#section1)<br>
2. [DataFusion Long](#section2)<br>
3. [DataFusion Wide](#section3)<br>
4. [Gráficos](#section4)<br>
5. [Escrever](#section5)<br>
6. [Visualizar](#section6)

# O que é preciso substituir:

1. Data Cleaning
   1. [Directória e nº bases Ninjas](#section1.1)
   2. [Directória e nº bases Delta](#section1.2)


---

<a id="section1"></a>
# <center>1. Data Cleaning</center>
[Voltar ao índice](#section0)<br>


In [226]:
''' Ler os dicionários previamente criados para poder usar os códigos de produto e de loja em vez dos nomes '''

mapa_produtos = ler_json("dicionário_produtos")
mapa_produtos = {int(key): value for key, value in mapa_produtos.items()}

mapa_lojas = ler_json("dicionário_lojas")
mapa_lojas = {int(key): value for key, value in mapa_lojas.items()}

mapa_produtos2 = ler_json("dicionário_produtos_23")
mapa_produtos2 = {int(key): value for key, value in mapa_produtos2.items()}   

In [289]:
produtos

['CAFÉ DELTA Q MYTHIQ 80CAP',
 'CAFÉ DELTA Q EPIQ 80CAP',
 'CAFÉ DELTA Q QALIDUS 80CAP',
 'CAFÉ DELTA Q QHARACTER 80CAP',
 'CAFÉ GINGA 10CAP',
 'CAFÉ BELLISSIMO AROMA INTENSO 30CAP',
 'CEVADA SOLÚVEL DELTA FRASCO 200G',
 'CAFÉ SOLÚVEL DELTA FRASCO 200G',
 'CAFÉ BELLISSIMO INTENSO 10 CAP',
 'CAFÉ BELLISSIMO ORIGINALE 10 CAP',
 'CAFFELATTE GO CHILL DELTA 230 ML',
 'CAPPUCCINO GO CHILL DELTA 230 ML',
 'Go Chill Double Expresso',
 'CARAMEL MACCHIATO GO CHILL DELTA 230ML',
 'LATTE MACCHIATO GO CHILL S/LAC 230ML']

- # `Ninjas`

> <font color=blue>Ler</font>
<a id="section1.1"></a>

In [227]:
%%time
# Ler o ficheiro long com Stocks e Fornecimento

directória = "D:\\B&N Dados\\Delta\\Padrão\\Padrão_08_2023\\"
num_bases = 8

prefixo = "1_Ninjas"
extensão = ".xlsx" #xlsb


dataframes = []

for i in range(1, num_bases + 1):
    file_path = f"{directória}{prefixo}1{extensão}"
    df = pd.read_excel(file_path, sheet_name=f'{i}')
    dataframes.append(df)
    
dfNinjas = pd.concat(dataframes, ignore_index=True)
dfNinjas = dfNinjas.rename(columns={' Data de Resposta': 'DATA', "Código da loja":"STORE", "Nome da Loja":"Loja"})

#dfNinjas = dfNinjas.rename(columns=lambda x: mapa_produtos2.get(x, x))

"""Datetime"""

# se xlsx
dfNinjas['DATA'] = pd.to_datetime(dfNinjas['DATA'], format='%d-%m-%Y') 

# se xlsb
#dfNinjas['DATA'] = pd.to_datetime(dfNinjas['DATA'], unit='d', origin=datetime.datetime(1899, 12, 30)) 

CPU times: total: 266 ms
Wall time: 272 ms


In [228]:
dfNinjas = dfNinjas.rename(columns={"Delta Q Mythiq 80":"CAFÉ DELTA Q MYTHIQ 80CAP","Delta Q EpiQ 80":"CAFÉ DELTA Q EPIQ 80CAP","Delta Q Qalidus 80":"CAFÉ DELTA Q QALIDUS 80CAP","Delta Q Qharacter 80":"CAFÉ DELTA Q QHARACTER 80CAP","Delta Q Ginga Cx. 10UN":"CAFÉ GINGA 10CAP","Bellissimo Aroma Intenso Pack 30 uni":"CAFÉ BELLISSIMO AROMA INTENSO 30CAP","Delta Cevada 200g":"CEVADA SOLÚVEL DELTA FRASCO 200G","Delta Cafés Solúvel 200g":"CAFÉ SOLÚVEL DELTA FRASCO 200G","Café Moído Bellissimo Intenso":"CAFÉ BELLISSIMO INTENSO 10 CAP","Café Moído Bellissimo Originale":"CAFÉ BELLISSIMO ORIGINALE 10 CAP"})

- Lista de produtos e lista de lojas

# <font color=red>É preciso definir melhor o formato do ficheiro dos ninjas para se saber os produtos, preciso de perceber melhor como é gerado na aplicação</font>

In [229]:
# Produtos
ficheiro_Produtos = dfNinjas.columns.tolist()
df_produtos = pd.DataFrame(ficheiro_Produtos[3:], columns=["DESC_ARTIGO"]) # Se não houver hora
#df_produtos = pd.DataFrame(ficheiro_Produtos[4:], columns=["DESC_ARTIGO"]) # Se houver hora

# Lojas
df_lojas = pd.DataFrame(dfNinjas["STORE"].unique().tolist(), columns=["STORE"])


'''Colocar em listas para serem chamáveis'''

lojas = df_lojas["STORE"].tolist()
produtos = df_produtos["DESC_ARTIGO"].tolist()
numprod = len(produtos)

[Escrever](#section5)

---

<a id="section2.1"></a>
- #  `Delta`

> <font color=blue>Ler</font>
<a id="section1.2"></a>

In [230]:
%%time
# Ler o ficheiro long com Stocks e Fornecimento

directória = "D:\\B&N Dados\\Delta\\Padrão\\Padrão_08_2023\\"
extensão = ".xlsb"
#extensão = ".xlsx"
num_bases = 4

prefixo = "2_Delta"

dataframes = []

for i in range(1, num_bases + 1):
    file_path = f"{directória}{prefixo}{i}{extensão}"
    df = pd.read_excel(file_path)
    dataframes.append(df[['DATA', 'EAN', 'DESC_ARTIGO', 'STORE', "STORE_NAME", 
                         "SOH", "INTRANSIT", "EXPECTED", "PRES_STOCK", "VND (D-1)"]])
    
dfDelta=pd.concat(dataframes, ignore_index=True)

"""Datetime"""

# se xlsx
#dfDelta['DATA'] = pd.to_datetime(dfDelta['DATA'], format='%d-%m-%Y') 

# se xlsb
dfDelta['DATA'] = pd.to_datetime(dfDelta['DATA'], unit='d', origin=datetime.datetime(1899, 12, 30)) 

CPU times: total: 22.5 s
Wall time: 22.5 s


> - **Definir os produtos e lojas em causa**

In [231]:
# Estabelecer o Dataframe mais limitado
dfDelta_Limpo = dfDelta[dfDelta['STORE'].isin(lojas) & dfDelta['DESC_ARTIGO'].isin(produtos)].copy()

# Mudar nomes
dfDelta_Limpo = dfDelta_Limpo.rename(columns={"VND (D-1)": "SELLOUT_1_Dias_Antes", "SOH": "STOCK_1_Dias_Antes"})

> - **Criar colunas de Stock e Sellout para o próprio dia**

In [232]:
# Organizar para atingir os valores relevantes
dfDelta_Limpo = dfDelta_Limpo.sort_values(by=["STORE", "DESC_ARTIGO", "DATA"])

#Agrupar por loja e produto para não haver valores errados 
dfDelta_Limpo["SELLOUT"] = dfDelta_Limpo.groupby(["STORE", "DESC_ARTIGO"])["SELLOUT_1_Dias_Antes"].shift(-1)
dfDelta_Limpo["STOCK"] = dfDelta_Limpo.groupby(["STORE", "DESC_ARTIGO"])["STOCK_1_Dias_Antes"].shift(-1)

> - **Retirar os dias que não têm interesse (último dia)**

In [233]:
# O dia final foi utilizado para conseguir dados do dia para stocks e sellouts mas não é dia de missão
segunda = dfDelta_Limpo['DATA'].unique()

dfDelta_Limpo = dfDelta_Limpo.loc[dfDelta_Limpo['DATA'] < segunda[-1]]
dfDelta_Limpo_Org = dfDelta_Limpo.iloc[:, np.r_[:5,11,6:9,10]]


### <center><font color=green>   Ainda fazer

> - **Id**

In [234]:
# Extract the day and month components from the 'datetime' column
dfNinjas['day'] = pd.to_datetime(dfNinjas['DATA']).dt.day
dfNinjas['month'] = pd.to_datetime(dfNinjas['DATA']).dt.month

# Create a new column combining day and month
dfNinjas['day_month'] = dfNinjas['day'].astype(str) + '-' + dfNinjas['month'].astype(str)

# Assign unique IDs to each unique day-month combination
dfNinjas['id'], _ = pd.factorize(dfNinjas['day_month'])

dfNinjas = dfNinjas.drop(columns=['day', 'month', 'day_month'])

> - **Número de produtos presentes**

In [235]:
# Criar coluna com o número dos produtos em causa presentes no instante

dfNinjas["Num_Produtos_Audit"] = dfNinjas[produtos].sum(axis=1)

dfNinjas.loc[(dfNinjas["Num_Produtos"] == 0) & (dfNinjas[produtos[1]].isna()), "Num_Produtos"] = np.nan

> - **Vendedor**

In [236]:
# Ler ficheiro
dfVendedor=pd.read_excel("D:\\B&N Dados\\Delta\\Vendedor2.xlsx", sheet_name = "Lojas Sonae para o desafio")
dfVendedor = dfVendedor.rename(columns={"Cód. Loja":"STORE"})

# Criar coluna de reposição
dfNinjas = pd.merge(dfNinjas, dfVendedor[["STORE","Vendedor"]], how="left", on = "STORE")

> - **Reposição**

In [237]:
# Ler o ficheiro com os nomes das lojas que fazem reposição
dfRepos=pd.read_excel("D:\\B&N Dados\\Delta\\Reposição_Sonae_Código.xlsx")

# Criar coluna que determina quais lojas têm reposição
dfNinjas['Repos'] = [1 if val in dfRepos['STORE'].values else 0 for val in dfNinjas['STORE']]

## <center><font color=orange>Output DataCleaning

In [238]:
[f"{index}: {column}" for index, column in enumerate(dfNinjas)]

['0: Loja',
 '1: STORE',
 '2: DATA',
 '3: CAFÉ DELTA Q MYTHIQ 80CAP',
 '4: CAFÉ DELTA Q EPIQ 80CAP',
 '5: CAFÉ DELTA Q QALIDUS 80CAP',
 '6: CAFÉ DELTA Q QHARACTER 80CAP',
 '7: CAFÉ GINGA 10CAP',
 '8: CAFÉ BELLISSIMO AROMA INTENSO 30CAP',
 '9: CEVADA SOLÚVEL DELTA FRASCO 200G',
 '10: CAFÉ SOLÚVEL DELTA FRASCO 200G',
 '11: CAFÉ BELLISSIMO INTENSO 10 CAP',
 '12: CAFÉ BELLISSIMO ORIGINALE 10 CAP',
 '13: CAFFELATTE GO CHILL DELTA 230 ML',
 '14: CAPPUCCINO GO CHILL DELTA 230 ML',
 '15: Go Chill Double Expresso',
 '16: CARAMEL MACCHIATO GO CHILL DELTA 230ML',
 '17: LATTE MACCHIATO GO CHILL S/LAC 230ML',
 '18: id',
 '19: Num_Produtos',
 '20: Vendedor',
 '21: Repos']

In [239]:
dfNinjas = dfNinjas.iloc[:, np.r_[5+numprod,0:5+numprod,6+numprod]].copy() # Se não houver hora
#dfNinjas1 = dfNinjas.iloc[:, np.r_[6+numprod,0:6+numprod,7+numprod]].copy() # Se houver hora
dfDelta = dfDelta_Limpo_Org.copy()

---

<a id="section2"></a>
# <center> 2. DataFusion Long (1)</center>
[Voltar ao índice](#section0)

> - __dfNinjas__ já definido
> - __dfDelta__ já definido
> - __produtos__ já definidos

- #  `dfFinal1`

In [240]:
resto = dfNinjas.columns.difference(produtos)
# Passar ficheiro ninjas para long
dfNinjasLong = dfNinjas.melt(id_vars=resto, value_vars=produtos, var_name='DESC_ARTIGO', value_name='NinjaInfo')


#Mergir
dfMeio = pd.merge(dfNinjasLong, dfDelta, how="left", on = ["DATA","STORE", "DESC_ARTIGO"]) 
#dfMeio = dfMeio.drop(columns=["Loja"])

dfFinal1=dfMeio.copy()

> STK

In [243]:
# Definir coluna de STK (soma de stocks com stock em trânsito)

dfFinal1["STK"] = dfFinal1["STOCK"] + dfFinal1["INTRANSIT"] + dfFinal1["EXPECTED"]

> Balanço

In [244]:
# Definir coluna de Balanço (razão entre sellout e stock total)

dfFinal1["Balance"] = dfFinal1["SELLOUT"] / dfFinal1["STK"]

> Rotura

In [245]:
# Definir coluna de rotura (se stock menor ou igual a 0 e existe Linear)

dfFinal1["ROTURA"] = np.where((dfFinal1["STOCK"] <= 0) & (dfFinal1["PRES_STOCK"] > 0), 1, 0)

>Sinal

In [246]:
# Categorias possíveis
mapping = {1: "Presente com Stock",
           2: "Ausente com Stock",
           3: "Presente sem Stock",
           4: "Ausente sem Stock"}
#            5: "Presente sem Registo",
#            6: "Ausente sem Registo"}

# Definir coluna de sinal
dfFinal1["Sinal"] = np.where((dfFinal1["STOCK"] > 0) & (dfFinal1["NinjaInfo"] == 1), 1,
                     np.where((dfFinal1["STOCK"] > 0) & (dfFinal1["NinjaInfo"] == 0), 2,
                     np.where((dfFinal1["STOCK"] <= 0) & (dfFinal1["NinjaInfo"] == 1), 3,
                     np.where((dfFinal1["STOCK"] <= 0) & (dfFinal1["NinjaInfo"] == 0), 4,
                     np.where((dfFinal1["STOCK"].isna()), np.nan, np.nan)))))

# Substituir números pelas expressões escolhidas
dfFinal1["Sinal"] = dfFinal1["Sinal"].map(mapping)
#dfFinal1["NinjaInfo"] = np.where(dfFinal1["STOCK"].isna(), np.nan, dfFinal1["NinjaInfo"])

> Ciclos

In [247]:
dfFinal1["Ciclos"]=dfFinal1["STOCK"]/dfFinal1["PRES_STOCK"]

> Adequação

In [248]:
dfFinal1["Adequacao"] = np.where(
                        dfFinal1["Ciclos"] > 1.1,
                        "Stock Suficiente",
                       np.where(
                        (dfFinal1["Ciclos"] <= 1.1) & (dfFinal1["INTRANSIT"] + dfFinal1["EXPECTED"] + dfFinal1["STOCK"] >= dfFinal1["PRES_STOCK"]),
                        "Stock Insuf c Forn Adequado",
                       np.where(
                        (dfFinal1["Ciclos"] <= 1.1) & (dfFinal1["INTRANSIT"] + dfFinal1["EXPECTED"] + dfFinal1["STOCK"] < dfFinal1["PRES_STOCK"]),
                        "Stock Insuf c Forn Desadequado",
                        "")))

> Dias para a rotura de stock

In [249]:
dfFinal1["Dias_para_Rotura"] = dfFinal1["STOCK"] / dfFinal1["SELLOUT"]

> Dias para a rotura de prateleira

In [250]:
dfFinal1["Dias_para_Rotura_Linear"] = dfFinal1["PRES_STOCK"] / dfFinal1["SELLOUT"]

> Roturas de prateleira (meh)

- ###  `Organizar`

## <center><font color=orange>Output DataFusion Long

In [251]:
df1 = dfFinal1.iloc[:, np.r_[5,0:2,4,7:16,2:4,6,16:19,19:24]].copy()

In [252]:
[f"{index}: {column}" for index, column in enumerate(df1.columns.tolist())]

['0: Vendedor',
 '1: DATA',
 '2: Loja',
 '3: STORE',
 '4: DESC_ARTIGO',
 '5: NinjaInfo',
 '6: EAN',
 '7: STORE_NAME',
 '8: STOCK',
 '9: INTRANSIT',
 '10: EXPECTED',
 '11: PRES_STOCK',
 '12: SELLOUT',
 '13: Num_Produtos',
 '14: Repos',
 '15: id',
 '16: STK',
 '17: Balance',
 '18: ROTURA',
 '19: Sinal',
 '20: Ciclos',
 '21: Adequacao',
 '22: Dias_para_Rotura',
 '23: Dias_para_Rotura_Linear']

[Escrever](#section5) <br>
[Visualizar](#section6)

<a id="section3"></a>
# <center>3. DataFusion Wide (2)</center>
[Voltar ao índice](#section0)<br>

> - __dfNinjas__ já definido
> - __dfDelta__ já definido
> - __produtos__ já definidos

- #  `dfFinal2`

In [253]:
# Dataframe mergido no fim
dfMeio2 = dfNinjas.copy()


for coluna in produtos:       
    
    #Cada dfStocks é um dataframe com o produto específico
    dfStocks=dfDelta[dfDelta["DESC_ARTIGO"]==coluna][["DATA", "STORE", "STOCK", "PRES_STOCK", "INTRANSIT", "EXPECTED", "SELLOUT"]]   #Dados Ninjas, seleccionar colunas do café específico
 
    #Juntar dados de acordo com o dia e a loja
    dfMeio2=pd.merge(dfMeio2, dfStocks, how="left", on = ["DATA","STORE"])  
    
    #Mudar nomes para ser adaptado a cada produto
    dfMeio2=dfMeio2.rename(columns={"STOCK": "STOCK %s" % coluna, "PRES_STOCK":"PRESLINEAR %s" % coluna, "INTRANSIT":"INTRANSIT %s" % coluna, "EXPECTED":"EXPECTED %s" % coluna, "SELLOUT":"SELLOUT %s" % coluna})                       #Nomear coluna nova
    
dfFinal2=dfMeio2.copy()

---

> STK

In [254]:
for i in produtos:

    transito = "INTRANSIT %s" % i
    esperado = "EXPECTED %s" % i
    stock  = "STOCK %s" % i
    
    stk = "STK %s" %i
      
    dfFinal2[stk] = dfFinal2[stock] + dfFinal2[esperado] + dfFinal2[transito]

> Balanço

In [255]:
for i in produtos:
    stk = "STK %s" %i
    #msa = "MSA %s" %i
    sellout = "SELLOUT %s" %i
    
    balance = "BALANCE %s" %i
    
    dfFinal2[balance] = dfFinal2[sellout] / dfFinal2[stk]
    
    

> Rotura

In [256]:
for i in produtos:
    rotura  = "ROTURA %s" % i
    stock  = "STOCK %s" % i
    preslinear = "PRESLINEAR %s" % i
    
    dfFinal2[rotura] = np.where((dfFinal2[stock] > 0) & ~(dfFinal2[i].isna()), 0, 
                      np.where((dfFinal2[stock] <= 0) & (dfFinal2[preslinear] > 0) & ~(dfFinal2[i].isna()), 1, 
                      np.nan))


> Pré-Rotura

In [257]:
for i in produtos:
    stock  = "STOCK %s" % i
    preslinear = "PRESLINEAR %s" % i
    pre_rotura= "PRE-ROTURA %s" % i
    
    dfFinal2[pre_rotura] = (dfFinal2[stock] < dfFinal2[preslinear]).astype(int)

>Sinal

In [258]:
#dfFinal2["Sinal"]= pd.Series(dtype='int')

# Define the mappings of numbers to strings
mapping = {1: "Presente com Stock",
           2: "Ausente com Stock",
           3: "Presente sem Stock",
           4: "Ausente sem Stock"}
#            5: "Presente sem Registo",
#            6: "Ausente sem Registo"}

for i in produtos:
    sinal = "SINAL %s" % i
    stock  = "STOCK %s" % i
    ninjainfo = i
    dfFinal2[sinal] = np.where((dfFinal2[stock] > 0) & (dfFinal2[i] == 1), 1,
                     np.where((dfFinal2[stock] > 0) & (dfFinal2[i] == 0), 2,
                     np.where((dfFinal2[stock] <= 0) & (dfFinal2[i] == 1), 3,
                     np.where((dfFinal2[stock] <= 0) & (dfFinal2[i] == 0), 4,
                     np.nan))))
#                      np.where((dfFinal2[stock].isna()) & (dfFinal2[i] == 1), 5,
#                      np.where((dfFinal2[stock].isna()) & (dfFinal2[i] == 0), 6,
#                      np.nan))))))


    # Map the numbers to the corresponding strings
    dfFinal2[sinal] = dfFinal2[sinal].map(mapping)
    dfFinal2[ninjainfo] = np.where(dfFinal2[stock].isna(), np.nan, dfFinal2[i])

# 1: Há Stock e Sinal Positivo Ninja "Verdadeiramente Correcto"
# 2: Há Stock e Sinal Negativo Ninja "Falso Positivo"
# 3: Não há Stock e Sinal Positivo Ninja "Falso Negativo"
# 4: Não há Stock e Sinal Negativo Ninja "Rejeição Correcta"

> Ciclos

In [259]:
for i in produtos:
    ciclos = "CICLO %s" % i
    stock  = "STOCK %s" % i
    linear = "PRESLINEAR %s" % i
    dfFinal2[ciclos]=dfFinal2[stock]/dfFinal2[linear]

C:\Users\joao_\AppData\Local\Temp\ipykernel_2260\3917726191.py:5: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

C:\Users\joao_\AppData\Local\Temp\ipykernel_2260\3917726191.py:5: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

C:\Users\joao_\AppData\Local\Temp\ipykernel_2260\3917726191.py:5: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented fra

> Adequação

In [260]:
dfFinal2 = dfFinal2.copy()
# Define the mappings of numbers to strings
mapping = {1: "Stock Suficiente",
           2: "Stock Insuf c Forn Adequado",
           3: "Stock Insuf c Forn Desadequado"}

for i in produtos:
    adequa = "ADEQUACAO %s" % i
    ciclos = "CICLO %s" % i
    transito = "INTRANSIT %s" % i
    esperado = "EXPECTED %s" % i
    stock  = "STOCK %s" % i
    linear = "PRESLINEAR %s" % i
    
    dfFinal2[adequa] = np.where(dfFinal2[ciclos] > 1.1, 1,
                      np.where((dfFinal2[ciclos] <= 1.1) & (dfFinal2[transito] + dfFinal2[esperado] + dfFinal2[stock] >= dfFinal2[linear]), 2,
                      np.where((dfFinal2[ciclos] <= 1.1) & (dfFinal2[transito] + dfFinal2[esperado] + dfFinal2[stock] < dfFinal2[linear]), 3, 
                      np.nan)))
    
    # Map the numbers to the corresponding strings
    dfFinal2[adequa] = dfFinal2[adequa].map(mapping)

- Dias para a rotura

In [261]:
for i in produtos:
    stock  = "STOCK %s" % i
    #msa = "MSA %s" %i
    sellout = "SELLOUT %s" % i
    dias_rotura = "Dias_para_Rotura %s" % i
    
    dfFinal2[dias_rotura] = dfFinal2[stock]/dfFinal2[sellout]

- Dias para a rotura de prateleira

In [262]:
for i in produtos:
    linear = "PRESLINEAR %s" % i
    #msa = "MSA %s" %i
    sellout = "SELLOUT %s" % i
    dias_rotura_prat = "Dias_para_Rotura_Linear %s" % i
    
    dfFinal2[dias_rotura_prat] = dfFinal2[linear] / dfFinal2[sellout]

---

# `2022`

In [263]:
# Ler o ficheiro long com Stocks e Fornecimento
dfDelta_2022=pd.read_excel("D:\\B&N Dados\\Delta\\Padrão\\Padrão_08_2023\\2_Delta_2022Id.xlsx", sheet_name="Sheet1")

#### dfFinal2_2022

In [264]:
# Dataframe mergido no fim
dfNinjas_2022=dfNinjas.copy()


for coluna in produtos:       
    
    #Cada dfStocks é um dataframe com o produto específico
    dfStocks=dfDelta_2022[dfDelta_2022["DESC_ARTIGO"]==coluna][["id", "STORE", "STOCK", "PRES_STOCK", "INTRANSIT", "EXPECTED", "SELLOUT"]]   #Dados Ninjas, seleccionar colunas do café específico
    
    #Juntar dados de acordo com o dia e a loja
    dfNinjas_2022=pd.merge(dfNinjas_2022, dfStocks, how="left", on = ["id","STORE"])  
    
    #Mudar nomes para ser adaptado a cada produto
    dfNinjas_2022=dfNinjas_2022.rename(columns={"STOCK": "STOCK22 %s" % coluna, "PRES_STOCK":"PRESLINEAR22 %s" % coluna, "INTRANSIT":"INTRANSIT22 %s" % coluna, "EXPECTED":"EXPECTED22 %s" % coluna, "SELLOUT":"SELLOUT22 %s" % coluna})                       #Nomear coluna nova

dfFinal2_2022 = dfNinjas_2022

## <center><font color=orange>Output DataFusion Wide

In [265]:
df2 = dfFinal2.copy()
df2_2022 = dfFinal2_2022.copy()

[Escrever](#section5) <br>
[Visualizar](#section6)

---

<a id="section4"></a>
# <center>4. Gráficos (3)</center>
[Voltar ao índice](#section0)<br>

> - __produtos__ já definidos

---

- #  `dfFinal3`

In [266]:
dfFinal3 = df2.copy()

# Tabelas

# - <span style="color:green">Classic:</span>  Proporção de presença por produto

In [267]:
# Média tirando os missing values
df_Med = dfFinal3[produtos].mean(numeric_only=True)#.groupby('Altura_do_Dia').mean()

# Passar para dataframe
df_Classic = df_Med.to_frame()

# Mudanças para ficar legível
df_Classic = df_Classic.reset_index()
df_Classic = df_Classic.rename(columns={'index': 'Produto', 0: 'Frequencia'})

In [268]:
import plotly.express as px
fig = px.bar(df_Classic, x='Produto', y='Frequencia', title='Classic')
fig.show()

- ###  <span style="color:BLUE">Com e sem Reposição</span>

In [269]:
# Definir se os produtos têm reposição

# Sim
df_ComRep = dfFinal3[dfFinal3["Repos"] == 1]

# Não
df_SemRep = dfFinal3[dfFinal3["Repos"] == 0]

>> <font color=blue>Com</font>

In [270]:
# Proporção
serie_MedRep = df_ComRep[produtos].mean(numeric_only=True)

# Para df
df_Classic_ComRepo = serie_MedRep.to_frame()

# Mudanças para ficar legível
df_Classic_ComRepo = df_Classic_ComRepo.reset_index()
df_Classic_ComRepo = df_Classic_ComRepo.rename(columns={'index': 'Produto', 0: 'Frequencia'})

>> <font color=blue>Sem

In [271]:
# Proporção
serie_MedSemRep = df_SemRep[produtos].mean(numeric_only=True)

# Proporção
df_Classic_SemRepo = serie_MedSemRep.to_frame()

# Mudanças para ficar legível
df_Classic_SemRepo = df_Classic_SemRepo.reset_index()
df_Classic_SemRepo = df_Classic_SemRepo.rename(columns={'index': 'Produto', 0: 'Frequencia'})

---

# - <font color=green> Final</font>: Proporção de presença por loja

In [306]:
# com reposição
dfFinal3["Num_ProdutosRep"] = np.nan
dfFinal3.loc[dfFinal3['Repos'] == 1, 'Num_ProdutosRep'] = dfFinal3["Num_Produtos"]


# sem reposição
dfFinal3["Num_ProdutosNRep"] = np.nan
dfFinal3.loc[dfFinal3['Repos'] == 0, 'Num_ProdutosNRep'] = dfFinal3["Num_Produtos"]

# Coluna para ter o número de produtos contratados pela marca
dfFinal3["Num_Produtos"] = dfFinal3[produtos].sum(axis=1)
dfFinal3["Produtos_Contratados"] = dfFinal3[produtos].notna().sum(axis=1)

In [307]:
'''Fazer colunas de proporção de produtos presentes em loja, depois especificando para quando têm e não têm reposição'''

df_NumProd=dfFinal3[["Loja","Num_Produtos","Num_ProdutosRep","Num_ProdutosNRep"]].copy()
df_NumProd["Prop_Produtos"] = (df_NumProd["Num_Produtos"] / dfFinal3["Produtos_Contratados"])*100
df_NumProd["Prop_ProdutosRep"] = (df_NumProd["Num_ProdutosRep"] / dfFinal3["Produtos_Contratados"])*100
df_NumProd["Prop_ProdutosNRep"] = (df_NumProd["Num_ProdutosNRep"] / dfFinal3["Produtos_Contratados"])*100

In [308]:
# Acrescentar à base original
dfFinal3["Prop_Produtos"] = df_NumProd["Prop_Produtos"]

In [309]:
df_NumProd

,Loja,Num_Produtos,Num_ProdutosRep,Num_ProdutosNRep,Prop_Produtos,Prop_ProdutosRep,Prop_ProdutosNRep
0,Continente Amadora,10.0,10.0,NaN,100.0,100.0,NaN
1,Continente Antas,10.0,10.0,NaN,100.0,100.0,NaN
2,Continente Aveiro,9.0,9.0,NaN,90.0,90.0,NaN
3,Continente Braga,10.0,10.0,NaN,100.0,100.0,NaN
4,Continente Braga 0va Arcada,10.0,NaN,10.0,100.0,NaN,100.0
...,...,...,...,...,...,...,...
133,Continente Matosinhos,4.0,4.0,NaN,100.0,100.0,NaN
134,Continente Modelo Albufeira,3.0,NaN,3.0,100.0,NaN,100.0
135,Continente Modelo Portalegre,3.0,NaN,3.0,100.0,NaN,100.0
136,Continente Santarém,4.0,NaN,4.0,100.0,NaN,100.0


---

# - <font color=green> Roturas</font>: Proporção de roturas por produto

In [275]:
df_RotMed = dfFinal3[["ROTURA %s" % produto for produto in produtos]].mean(numeric_only=True)

# Passar para dataframe
df_Rot = df_RotMed.to_frame()
df_Rot = df_Rot.reset_index()
df_Rot = df_Rot.rename(columns={'index': 'Produto', 0: 'Frequencia'})
df_Rot["Frequencia"] = df_Rot["Frequencia"]*100

## <center><font color=orange>Output DataFusion Wide

In [276]:
df2_2 = dfFinal3

#df_Classic
#df_Classic_ComRepo
#df_Classic_SemRepo
#df_Rot
#df_NumProd

[Visualizar](#section6)

---

In [277]:
produtos

['CAFÉ DELTA Q MYTHIQ 80CAP',
 'CAFÉ DELTA Q EPIQ 80CAP',
 'CAFÉ DELTA Q QALIDUS 80CAP',
 'CAFÉ DELTA Q QHARACTER 80CAP',
 'CAFÉ GINGA 10CAP',
 'CAFÉ BELLISSIMO AROMA INTENSO 30CAP',
 'CEVADA SOLÚVEL DELTA FRASCO 200G',
 'CAFÉ SOLÚVEL DELTA FRASCO 200G',
 'CAFÉ BELLISSIMO INTENSO 10 CAP',
 'CAFÉ BELLISSIMO ORIGINALE 10 CAP',
 'CAFFELATTE GO CHILL DELTA 230 ML',
 'CAPPUCCINO GO CHILL DELTA 230 ML',
 'Go Chill Double Expresso',
 'CARAMEL MACCHIATO GO CHILL DELTA 230ML',
 'LATTE MACCHIATO GO CHILL S/LAC 230ML']

In [278]:
df1[df1.DESC_ARTIGO == "CAFÉ BELLISSIMO ORIGINALE 10 CAP"].NinjaInfo.sum()

74.0

<a id="section5"></a>
# <center><u>Escrever</u></center>
[Voltar ao índice](#section0)

In [314]:
[f"{index}: {column}" for index, column in enumerate(df2_2)]

['0: Vendedor',
 '1: Loja',
 '2: STORE',
 '3: DATA',
 '4: CAFÉ DELTA Q MYTHIQ 80CAP',
 '5: CAFÉ DELTA Q EPIQ 80CAP',
 '6: CAFÉ DELTA Q QALIDUS 80CAP',
 '7: CAFÉ DELTA Q QHARACTER 80CAP',
 '8: CAFÉ GINGA 10CAP',
 '9: CAFÉ BELLISSIMO AROMA INTENSO 30CAP',
 '10: CEVADA SOLÚVEL DELTA FRASCO 200G',
 '11: CAFÉ SOLÚVEL DELTA FRASCO 200G',
 '12: CAFÉ BELLISSIMO INTENSO 10 CAP',
 '13: CAFÉ BELLISSIMO ORIGINALE 10 CAP',
 '14: CAFFELATTE GO CHILL DELTA 230 ML',
 '15: CAPPUCCINO GO CHILL DELTA 230 ML',
 '16: Go Chill Double Expresso',
 '17: CARAMEL MACCHIATO GO CHILL DELTA 230ML',
 '18: LATTE MACCHIATO GO CHILL S/LAC 230ML',
 '19: id',
 '20: Num_Produtos',
 '21: Repos',
 '22: STOCK CAFÉ DELTA Q MYTHIQ 80CAP',
 '23: PRESLINEAR CAFÉ DELTA Q MYTHIQ 80CAP',
 '24: INTRANSIT CAFÉ DELTA Q MYTHIQ 80CAP',
 '25: EXPECTED CAFÉ DELTA Q MYTHIQ 80CAP',
 '26: SELLOUT CAFÉ DELTA Q MYTHIQ 80CAP',
 '27: STOCK CAFÉ DELTA Q EPIQ 80CAP',
 '28: PRESLINEAR CAFÉ DELTA Q EPIQ 80CAP',
 '29: INTRANSIT CAFÉ DELTA Q EPIQ 80CA

### <center><font color=green>DataCleaning</font></center>

In [281]:
escrever_txt(df_produtos, "produtos")
escrever_txt(df_lojas, "lojas")

### <center><font color=green>DataFusion Long</font></center>

In [282]:
escrever_excel(df1, "Long_DataFusion")

### <center><font color=green>DataFusion Wide</font></center>

In [283]:
#escrever_excel(df2_1, "Wide_DataFusion_Inc")

escrever_excel(df2_2, "Wide_DataFusion")
escrever_excel(df2_2022, "Wide_DataFusion2022")

### <center><font color=green>Gráficos</font></center>


- Classic

In [310]:
# Normal
escrever_excel(df_Classic, "G_Classic")

# Com Reposição
escrever_excel(df_Classic_ComRepo, "G_Classic_Reposto")

# Sem Reposição
escrever_excel(df_Classic_SemRepo, "G_Classic_SemReposto")

- Roturas

In [311]:
escrever_excel(df_Rot, "G_Rotura")

- Número de produtos

In [312]:
escrever_excel(df_NumProd, "G_NumProdutos")

In [316]:
dfNinjas.shape

(138, 22)

In [318]:
len(df_NumProd[df_NumProd.Prop_Produtos==100])

71

In [319]:
71/138

0.5144927536231884

<a id="section6"></a>
# <center><u>Visualizar</u></center>
[Voltar ao índice](#section0)

- # `Long_DataFusion`

> Filtrar produto

In [287]:
@interact
def filtrar_produto(produto = list(df1.DESC_ARTIGO.unique())):
    
    return df1[df1.DESC_ARTIGO == produto].head() # Alterável

interactive(children=(Dropdown(description='produto', options=('CAFÉ DELTA Q MYTHIQ 80CAP', 'CAFÉ DELTA Q EPIQ…

> Filtrar loja

In [288]:
@interact
def filtrar_loja(loja = list(df1.Loja.unique())):
    
    return df1[df1.Loja == loja].head() # Alterável

interactive(children=(Dropdown(description='loja', options=('Continente Amadora', 'Continente Antas', 'Contine…

- # `Wide_DataFusion`

[Voltar ao índice](#section0)